In [1]:
!pip install tetris_gymnasium

In [2]:
import numpy as np
import random
import gymnasium as gym
import tetris_gymnasium
from tetris_gymnasium.envs.tetris import ActionsMapping

In [4]:
def printarray(array):
    for row in array:
        print(row)

In [5]:
board1 = [[0, 0, 0, 0, 0, 0, 8, 0, 0, 0],
        [0, 0, 0, 0, 8, 8, 8, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 1, 1, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]


#Finds the column heights for attached pieces
def find_column_heights(board):
    column_heights = [0] * 10
    for c in range(10):
        maxHeight = 0
        r = 19
        while r >= 0:
            if board[r][c] != 0 and is_attached(board, c, r):
                maxHeight = 20 - r
            r-=1
        column_heights[c] = maxHeight
    return column_heights



def is_valid(board, xindex, yindex):
    #Checking out of bounds
    if(yindex >= 20 or yindex < 0 or xindex < 0 or xindex >= 10):
        return False
    #Check if space is not a piece
    elif(board[yindex][xindex] == 0):
        return False
    else:
        return True


#Tested and Approved
#Checks if the piece which contains the position xindex yindex is connected
def is_attached(board, xindex, yindex):
    if(not is_valid(board, xindex, yindex)):
        return False
    queue = [(xindex, yindex)]
    visited = []
    while(len(queue) > 0):
        current_index = queue.pop()
        #print(current_index)
        visited.append(current_index)
        x = current_index[0]
        y = current_index[1]
        if y == 19:
            return True
        if(is_valid(board, x, y-1) and not ((x, y-1) in visited)):
            queue.append((x, y-1))
        if(is_valid(board, x+1, y) and not ((x+1, y) in visited)):
            queue.append((x+1, y))
        if(is_valid(board, x-1, y) and not ((x-1, y) in visited)):
            queue.append((x-1, y))
        if(is_valid(board, x, y+1) and not ((x, y+1) in visited)):
            queue.append((x, y+1))

    return False


#Used to find the coordinates of the active piece
def get_visited(board, xindex, yindex):
    #print(board[yindex][xindex])
    #printarray(board)
    #print(board[yindex])
    if(not is_valid(board, xindex, yindex)):
        #Stands for No Piece Found
        return "NPF"
    queue = [(xindex, yindex)]
    visited = []
    while(len(queue) > 0):
        current_index = queue.pop()
        #print(current_index)
        visited.append(current_index)
        x = current_index[0]
        y = current_index[1]
        if y == 19:
            return visited
        if(is_valid(board, x, y-1) and not ((x, y-1) in visited)):
            queue.append((x, y-1))
        if(is_valid(board, x+1, y) and not ((x+1, y) in visited)):
            queue.append((x+1, y))
        if(is_valid(board, x-1, y) and not ((x-1, y) in visited)):
            queue.append((x-1, y))
        if(is_valid(board, x, y+1) and not ((x, y+1) in visited)):
            queue.append((x, y+1))

    return visited

#is_attached_with_visited(board1, 5,1)



In [10]:
#Finds valid unattached pieces
def is_unattached(board, x_index, y_index):
    if(board[y_index][x_index] == 0):
        #print(f"There is no piece at {x_index, y_index}")
        return False
    elif(not is_attached(board, x_index, y_index)):
        return True
    return False

def find_active_piece(board):
    piece = 0
    xindex = 0
    yindex = 0
    #Row by row scan starting at the top to find the first piece which is unattached
    while(not is_unattached(board, xindex, yindex)):
        if(xindex == 9):
            xindex = 0
            yindex += 1
        else:
            xindex += 1
        if(yindex >= 20):
            #We do this so the is_unattached method doesn't have to deal with out of bounds
            break
    if(yindex < 20):
        piece = board[yindex][xindex]
    if(piece == 0):
        pass
        #print("No active piece found")
    piece_coordinates = get_visited(board, xindex, yindex)
    return piece_coordinates

def find_piece_width(board):
    piece_coordinates = find_active_piece(board)
    x_coordinates = []
    for coordinate in piece_coordinates:
        x_coordinates.append(coordinate[0])
    if(x_coordinates == ["N", "P", "F"]):
        return "No Piece Found"
    #print(x_coordinates)
    #print(x_coordinates == ["N", "P", "F"])
    piece_width = max(x_coordinates) - min(x_coordinates) + 1
    return piece_width

find_piece_width(board1)

def find_leftmost_index(board):
    piece_coordinates = find_active_piece(board)
    x_coordinates = []
    for coordinate in piece_coordinates:
        x_coordinates.append(coordinate[0])
    return min(x_coordinates)



In [76]:
import sys

import cv2
import gymnasium as gym
from tetris_gymnasium.envs import Tetris
# --- Initialize the Tetris Environment ---
computer_env = gym.make("tetris_gymnasium/Tetris", render_mode="ansi")
human_env = gym.make("tetris_gymnasium/Tetris", render_mode="human")


#Convert string board which is returned by env.render() to an array
def convert_strBoard_to_2dArray(board):
    array = []
    row = []
    for char in board:
        if char != "\n":
            if char == '.':
                row.append(0)
            else:
                row.append(int(char))
        else:
            array.append(row)
            row = []
    array.append(row)
    #printarray(array)
    return array

# --- Function to Extract Board Features ---
def get_board_features(board):
    """
    Extracts meaningful features from the Tetris board.
    """
    column_heights = [0] * 10
    for c in range(10):
        maxHeight = 0
        r = 19
        while r >= 0:
            if board[r][c] != 0:
                maxHeight = 20 - r
            r-=1
        column_heights[c] = maxHeight
    #print(column_heights)
    #printarray(board)
    max_height = max(column_heights)
    bumpiness = sum(abs(column_heights[i] - column_heights[i+1]) for i in range(9))
    holes = sum(1 for c in range(10) for r in range(column_heights[c]) if board[r][c] == 0)
    return np.array([max_height, holes, bumpiness])


def play_tetris():
    random_seed = int(np.random.rand()*1000)
    #print(random_seed)

    total_lines_cleared = 0
    total_reward = 0
    state = computer_env.reset(seed = random_seed)
    human_env.reset(seed = random_seed)
    done = False


    move_list = []

    while not done:
        board = computer_env.render()  # Get board as ansi values
        human_env.render()

        action = None
        while action is None:
            key = cv2.waitKey(1)

            if key == ord("a"):
                action = human_env.unwrapped.actions.move_left
            elif key == ord("d"):
                action = human_env.unwrapped.actions.move_right
            elif key == ord("s"):
                action = human_env.unwrapped.actions.move_down
            elif key == ord("w"):
                action = human_env.unwrapped.actions.rotate_counterclockwise
            elif key == ord("e"):
                action = human_env.unwrapped.actions.rotate_clockwise
            elif key == ord(" "):
                action = human_env.unwrapped.actions.hard_drop
                print(action)
            elif key == ord("q"):
                action = human_env.unwrapped.actions.swap
            elif key == ord("r"):
                print("Escaped Line 85")
                action = 0 #Dummy action (should only negligibly skew the data)
                random_seed = int(np.random.rand()*1000)
                human_env.reset(seed=random_seed)
                computer_env.reset(seed=random_seed)
                done = True
                sys.exit()
                break

            if (
                cv2.getWindowProperty(human_env.unwrapped.window_name, cv2.WND_PROP_VISIBLE)
                == 0
            ):
                sys.exit()
        #print(action)
        #print(type(board))
        board = convert_strBoard_to_2dArray(board)

        #printarray(board)
        #column_heights = find_column_heights(board)
        #print(f"column heights: {column_heights}")
        #piece_width = find_piece_width(board)
        computer_step = computer_env.step(action)  # Store step results in a variable
        step_result = human_env.step(action)
        #printarray(board)

        # Handle both 4-value and 5-value return cases
        if len(step_result) == 4:
            next_state, reward, done, info = step_result  # Old format
        else:
            next_state, reward, done, truncated, info = step_result  # New format
            done = done or truncated  # Ensure 'done' includes truncation
        #print(truncated)
        total_reward += reward
        #print(info['lines_cleared'])
        #if(info['lines_cleared'] > 0):
            #printarray(board)
            #print(next_state)
        piece_coordinates = find_active_piece(board)
        #Format the piece coordinates
        if(piece_coordinates == "NPF" or len(piece_coordinates) != 4):
            piece_coordinates = [(-1, -1), (-1, -1), (-1, -1), (-1, -1)]
        total_lines_cleared += info['lines_cleared']  # Track lines cleared

        move = {"board": board, "piece": piece_coordinates, "action": action, "reward": reward}
        move_list.append(move)

    #print(total_lines_cleared)
    print("Escaped Line 130")
    computer_env.close()
    human_env.close()
    cv2.destroyAllWindows() 
    print(total_lines_cleared)
    print(total_reward)
    return {"Lines Cleared" : total_lines_cleared, "Reward": total_reward, "Move List":move_list}

def train_model(episodes):
    history = []
    for i in range(episodes):
        history.append(play_tetris())
    return history




In [77]:
history = train_model(1)

5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
Escaped Line 130
32
549


In [18]:

def extract_current_features(board, piece):
    move_data = []
    for row in board:
        for element in row:
            move_data.append(element)
            #print(piece[0][0])
    for pixel in piece:
        for coordinate in pixel:
            #print(coordinate)
            move_data.append(coordinate)
    if(len(move_data) > 209):
        move_data = move_data[0:208]
    return move_data



def extract_features(history):
    data = []
    for episode in history:
        move_list = episode["Move List"]
        for move in move_list:
            move_data = []
            board = move["board"]
            piece = move["piece"]
            action = move["action"]
            #add each
            for row in board:
                for element in row:
                    move_data.append(element)
            #move_data.append("|") #Used for debugging, remove later
            for pixel in piece:
                for coordinate in pixel:
                    #print(coordinate)
                    move_data.append(coordinate)
            move_data.append(action)
            if(len(move_data) != 209):
                print(move_data)
                print()
            data.append(move_data)
    return data

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
#Random Forest

#Build random Forest
def build_rf(data, depth, samples):
    data = np.array(data)
    features = data[:, :-1] #Selecting all rows but only columns from the first until the second to last
    label = data[:, -1]        #Last column (this is the move which is the label)
    #Build, train, and validate the model
    x_train = features
    y_train = label
    #Build model
    
    clf_gini = DecisionTreeClassifier(criterion="gini", max_depth = depth, min_samples_leaf=samples, random_state = 42)
    #Train model
    clf_gini.fit(x_train, y_train)
    #print(x_train[0])
    #print(y_train[0])

    y_pred_train_gini = clf_gini.predict(x_train)
    # print the scores on training and test set
    #print('Training set score: {:.4f}'.format(clf_gini.score(x_train, y_train)))
    return clf_gini


def RF_prediction(rf, current_board, current_piece):
    current_features = extract_current_features(current_board, current_piece)

    if(len(current_features) != 208):
        #Imput missing values with -1
        for i in range(208-len(current_features)):
            current_features.append(-1)
    #Make Prediction
    current_features = [np.array(current_features)]
    #print(current_features)
    return rf.predict(current_features)[0]

    #print('Training set score: {:.4f}'.format(clf_gini.score(x_train, y_train)))



model = build_rf(extract_features(history), 10, 20)
RF_prediction(model, board1, find_active_piece(board1))

4

In [23]:
def rf_play_game(depth, samples):
    total_lines_cleared = 0
    total_reward = 0
    state = computer_env.reset()
    done = False

    #Build RF
    rf = build_rf(extract_features(history), depth,samples)

    while not done:
        board = computer_env.render()  # Get board as image
        
        #print(type(board))
        board = convert_strBoard_to_2dArray(board)
        #printarray(board)
        #print()
        active_piece = find_active_piece(board)
        if(active_piece == "NPF"):
            active_piece = [(-1,-1), (-1,-1), (-1,-1), (-1,-1)]
        action = RF_prediction(rf, board, active_piece)
        #print(action)
        #print(f"action: {action}")
        # Ensure the action is valid
        #action = actions.get(action)  # Map to an action from ActionsMapping
        step_result = computer_env.step(action)  # Store step results in a variable

        # Handle both 4-value and 5-value return cases
        if len(step_result) == 4:
            next_state, reward, done, info = step_result  # Old format
        else:
            next_state, reward, done, truncated, info = step_result  # New format
            done = done or truncated  # Ensure 'done' includes truncation
        total_reward += reward
        total_lines_cleared += info["lines_cleared"]
    printarray(board)
    return {"reward": total_reward, "lines_cleared": total_lines_cleared}



In [72]:
rf_play_game(20, 21)

[0, 0, 0, 0, 0, 0, 8, 0, 0, 0]
[0, 0, 0, 0, 8, 8, 8, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 3, 3, 0, 0, 6]
[0, 0, 7, 7, 0, 3, 3, 0, 6, 6]
[0, 8, 7, 0, 0, 5, 5, 0, 6, 0]
[0, 8, 7, 0, 5, 5, 0, 0, 2, 0]
[0, 8, 8, 0, 4, 4, 4, 2, 2, 0]
[7, 7, 0, 0, 0, 4, 0, 2, 2, 0]
[7, 0, 0, 0, 0, 4, 0, 2, 2, 6]
[7, 0, 0, 0, 0, 4, 4, 2, 6, 6]
[8, 0, 0, 0, 0, 4, 5, 5, 6, 0]
[8, 0, 0, 0, 0, 5, 5, 0, 2, 0]
[8, 8, 0, 0, 0, 3, 3, 0, 2, 0]
[7, 7, 0, 0, 0, 3, 3, 0, 2, 0]
[7, 0, 0, 0, 4, 4, 4, 0, 2, 6]
[7, 0, 0, 0, 0, 4, 0, 0, 6, 6]
[8, 0, 0, 0, 0, 3, 3, 0, 6, 0]
[8, 0, 0, 0, 0, 3, 3, 5, 5, 0]
[8, 8, 0, 0, 0, 0, 5, 5, 0, 0]


{'reward': 21, 'lines_cleared': 0}

In [ ]:
#Hyperparameter Tuning
for depth in range(10, 41, 5):
    for samples in range(1,71, 10):
        avg_reward = 0
        avg_lines_cleared = 0
        for i in range(50):
            game = rf_play_game(depth, samples)
            avg_reward += game["reward"]
        avg_reward /= 50
        print(f"Depth: {depth}, samples: {samples}, average reward: {avg_reward}" )

Depth: 10, samples: 1, average reward: 14.92
Depth: 10, samples: 11, average reward: 20.8
Depth: 10, samples: 21, average reward: 23.8
Depth: 10, samples: 31, average reward: 12.04
Depth: 10, samples: 41, average reward: 20.14
Depth: 10, samples: 51, average reward: 22.52
Depth: 10, samples: 61, average reward: 15.74
Depth: 15, samples: 1, average reward: 15.1
Depth: 15, samples: 11, average reward: 21.12
Depth: 15, samples: 21, average reward: 23.36
Depth: 15, samples: 31, average reward: 13.06
Depth: 15, samples: 41, average reward: 21.52
Depth: 15, samples: 51, average reward: 20.82
Depth: 15, samples: 61, average reward: 15.78
Depth: 20, samples: 1, average reward: 14.98
Depth: 20, samples: 11, average reward: 21.66
Depth: 20, samples: 21, average reward: 24.06
Depth: 20, samples: 31, average reward: 12.98
Depth: 20, samples: 41, average reward: 21.52
Depth: 20, samples: 51, average reward: 21.36
Depth: 20, samples: 61, average reward: 15.34
Depth: 25, samples: 1, average reward: 1

In [ ]:
lines_cleared = 0
for i in range(250):
    lines_cleared += rf_play_game(20,21)["lines_cleared"]
lines_cleared/250

0.016

In [ ]:
copy = history
copy.append(train_model(1))
copy

[{'Lines Cleared': 66,
  'Reward': 1097,
  'Move List': [{'board': [[0, 0, 0, 0, 0, 5, 5, 0, 0, 0],
     [0, 0, 0, 0, 5, 5, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
    'piece': [(5, 0), (5, 1), (4, 1), (6, 0)],
    'action': 2,
    'reward': 0},
   {'board': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 5, 5, 0, 0

In [ ]:

for e in copy:
    for a in e:
        for b in a:
            print(len(b))

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
13
6
9
13
6
9
13
6
9
